In [21]:
from pyhive import hive

conn = hive.Connection(
    host="localhost",
    port=10000,
    username="maria_dev",
    database="foodmart",
    auth="NONE"
)

cursor = conn.cursor()
cursor.execute("SHOW TABLES")
for result in cursor.fetchall():
    print(result)


('account',)
('agg_c_10_sales_fact_1997',)
('agg_c_14_sales_fact_1997',)
('agg_c_special_sales_fact_1997',)
('agg_g_ms_pcat_sales_fact_1997',)
('agg_l_03_sales_fact_1997',)
('agg_l_04_sales_fact_1997',)
('agg_l_05_sales_fact_1997',)
('agg_lc_06_sales_fact_1997',)
('agg_lc_100_sales_fact_1997',)
('agg_ll_01_sales_fact_1997',)
('agg_pl_01_sales_fact_1997',)
('category',)
('currency',)
('customer',)
('days',)
('department',)
('employee',)
('employee_closure',)
('expense_fact',)
('inventory_fact_1997',)
('inventory_fact_1998',)
('position',)
('product',)
('product_class',)
('promotion',)
('region',)
('reserve_employee',)
('salary',)
('sales_fact_1997',)
('sales_fact_1998',)
('sales_fact_dec_1998',)
('store',)
('store_ragged',)
('time_by_day',)
('warehouse',)
('warehouse_class',)


In [17]:
from ucimlrepo import fetch_ucirepo

fraud = fetch_ucirepo(id=27)

print(fraud.data.features)

     A15    A14 A13 A12  A11 A10 A9    A8  A7  A6 A5 A4      A3     A2 A1
0      0  202.0   g   f    1   t  t  1.25   v   w  g  u   0.000  30.83  b
1    560   43.0   g   f    6   t  t  3.04   h   q  g  u   4.460  58.67  a
2    824  280.0   g   f    0   f  t  1.50   h   q  g  u   0.500  24.50  a
3      3  100.0   g   t    5   t  t  3.75   v   w  g  u   1.540  27.83  b
4      0  120.0   s   f    0   f  t  1.71   v   w  g  u   5.625  20.17  b
..   ...    ...  ..  ..  ...  .. ..   ...  ..  .. .. ..     ...    ... ..
685    0  260.0   g   f    0   f  f  1.25   h   e  p  y  10.085  21.08  b
686  394  200.0   g   t    2   t  f  2.00   v   c  g  u   0.750  22.67  a
687    1  200.0   g   t    1   t  f  2.00  ff  ff  p  y  13.500  25.25  a
688  750  280.0   g   f    0   f  f  0.04   v  aa  g  u   0.205  17.92  b
689    0    0.0   g   t    0   f  f  8.29   h   c  g  u   3.375  35.00  b

[690 rows x 15 columns]


In [25]:
import pandas as pd
df = pd.concat([fraud.data.features, fraud.data.target], axis=1)
df.to_csv('fraud.csv', index=False, header=False)